In [389]:
import pycurl, json, sys
import re
import pandas as pd
import numpy as np
import datetime as dt
from common_func import check_url
from BeautifulSoup import BeautifulSoup
import urllib2
import unidecode

In [321]:
pkls = !ls pkl/itunes*COMPLETE.pkl
podcastDf = pd.read_pickle(pkls[-1])
podcastDf.shape

(49930, 36)

In [322]:
# filter out podcasts with no itunes collection id
podcastDf = podcastDf[np.isfinite(podcastDf['collectionId'])]
podcastDf.shape

(33227, 36)

In [323]:
# filter out podcasts without recent episodes
days_thresh = 45
thresh_date = dt.datetime.today() - dt.timedelta(days=days_thresh)

podcastDf['releaseDate'] = pd.to_datetime(podcastDf['releaseDate'])
podcastDf = podcastDf[podcastDf.releaseDate > thresh_date]
podcastDf.shape

(6273, 36)

In [324]:
# convert collectionId to int
podcastDf['collectionId'] = [int(x) for x in podcastDf['collectionId']]

In [98]:
# make sample for testing
testDf = podcastDf.sample(n=20)

In [205]:
baseUrl = 'https://itunes.apple.com/us/podcast/id'

In [173]:
# will store pycurl output
class Test:
   def __init__(self):
       self.contents = ''

   def body_callback(self, buf):
       self.contents = self.contents + buf

In [313]:
def run_curl(url):
    t = Test()
    c = pycurl.Curl()
    c.setopt(pycurl.URL, url)
    c.setopt(pycurl.HTTPHEADER, ['X-Apple-Store-Front: 143441-1,12', 'X-Apple-Tz: 3600'])
    c.setopt(pycurl.USERAGENT, 'iTunes/9.2.1 (Macintosh; Intel Mac OS X 10.5.8) AppleWebKit/533.16')
    c.setopt(pycurl.SSL_VERIFYHOST, 0)
    c.setopt(pycurl.SSL_VERIFYPEER, 0)
    c.setopt(pycurl.WRITEFUNCTION, t.body_callback)
    c.perform()
    return t

In [426]:
def clean_description(d):
    d = unidecode.unidecode(d)
    d = d.replace('\n', ' ')
    if re.findall(r'(.*) brought to you by.*', d):
       d = re.sub(r'brought to you by.*', '', d)
    if re.search(r'(.*) sponsored by.*', d):
       d = re.sub(r'sponsored by.*', '', d)
    return d

In [430]:
for ind, row in testDf.iterrows():
    itunesId = row['collectionId']
    scrapeUrl = baseUrl + str(itunesId)
    
    # get podcast summary
    t = run_curl(scrapeUrl)
    soup = BeautifulSoup(t.contents)
    p = soup.p
    if p:
        podcastSummary = soup.p.string
    else: # redirect
        newUrl = soup.findAll(text=re.compile(r'https'))
        newUrl = newUrl[0]
        newUrl = re.sub(r'&amp;', r'&', newUrl)
        t = run_curl(newUrl)
        soup = BeautifulSoup(t.contents)
        p = soup.p
        if p:
            podcastSummary = soup.p.string
        else:
            podcastSummary = np.nan
    
    # get episode descriptions
    episodeData = soup.findAll('button', kind='episode')
    try:
        episodeData = [clean_description(e['description']) for e in episodeData]
    except: # no description
        episodeData = np.nan

no data


In [432]:
itunesId

279821049

In [328]:
# get political gabfest data
scrapeUrl = baseUrl + '158004641'
t = run_curl(scrapeUrl)
soup = BeautifulSoup(t.contents)
newUrl = soup.findAll(text=re.compile(r'https'))
newUrl = newUrl[0]
newUrl = re.sub(r'&amp;', r'&', newUrl)
t = run_curl(newUrl)
soup = BeautifulSoup(t.contents)

In [422]:
# get savage lovecast data
scrapeUrl = baseUrl + '201376301'
t = run_curl(scrapeUrl)
soup = BeautifulSoup(t.contents)
newUrl = soup.findAll(text=re.compile(r'https'))
newUrl = newUrl[0]
newUrl = re.sub(r'&amp;', r'&', newUrl)
t = run_curl(newUrl)
soup = BeautifulSoup(t.contents)

In [424]:
episode_data = soup.findAll('button', kind='episode')

In [425]:
for e in episode_data:
    description = e['description']
    description = unidecode.unidecode(description)
    description = description.replace('\n', ' ')
    if re.findall(r'(.*) brought to you by.*', description):
       description = re.sub(r'brought to you by.*', '', description)
    if re.search(r'(.*) sponsored by.*', description):
       description = re.sub(r'sponsored by.*', '', description)
    print description

A young woman is about to embark on a three-way with her boyfriend and another man. How can she make sure the experience isn't demeaning?  A gay man has incest fantasies. How can he find a man willing to role play a father-figure?  A married woman and her husband prefer to use condoms. So why is everyone around her so smug and condescending about it?  On the Magnum, Dan speaks with Dr. Robert Garofalo about a father whose kid calls himself transmale, but often presents as female. They discuss the challenges parents face in keeping an open mind and an open heart as kids and teens figure themselves out.  206-302-2064  This episode is 
What to do when your drunken cousin starts blabbing about all the affairs she's having and all the cocaine she's snorting and all the traffic accidents she's having?  A professor is in anguish because she is about to sleep with a much younger, very good looking FORMER student. A couple of her (male) colleagues think this is a bad idea. Most of them (all wom

KeyError: 'description'

In [14]:
also_subcribed_ids = re.findall(r'adam-id="(\d+)" aria-label=', t.contents)

['268213039',
 '73329771',
 '294055449',
 '73330715',
 '216713308',
 '155974141',
 '124960485',
 '258723953',
 '278974813',
 '98746009']